In [12]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
import csv
import datetime
from datetime import date, timedelta

In [51]:
#path to datafiles
stat_url =  "static/data/fl_ev_stations.csv"
stat_df = pd.read_csv(stat_url)

In [18]:
#remove unecessary columns
stat_df = stat_df[['Fuel Type Code', 'Station Name', 'Street Address','Latitude', 'Longitude', 'Open Date', 'ZIP']]
stat_df.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,Open Date,ZIP
6,ELEC,City of Delray Beach - Banker's Row Parking Lot,200 NE 1st Ave,26.465314,-80.072190,6/15/2011,33444
8,ELEC,Firkins Nissan,1611 Cortez Rd W,27.462023,-82.578036,3/15/2011,34207
9,ELEC,Maus Nissan,937 S Suncoast Blvd,28.848733,-82.579470,3/15/2011,34448
10,ELEC,Deland Nissan,2600 S Woodland Blvd,28.983689,-81.300079,3/15/2011,32720
11,ELEC,Delray Nissan,2200 S Federal Hwy,26.435016,-80.072333,3/15/2011,33483


In [17]:
#drop rows with empty Values (NaN)
stat_df = stat_df.dropna()
stat_df.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,Open Date,ZIP
6,ELEC,City of Delray Beach - Banker's Row Parking Lot,200 NE 1st Ave,26.465314,-80.072190,6/15/2011,33444
8,ELEC,Firkins Nissan,1611 Cortez Rd W,27.462023,-82.578036,3/15/2011,34207
9,ELEC,Maus Nissan,937 S Suncoast Blvd,28.848733,-82.579470,3/15/2011,34448
10,ELEC,Deland Nissan,2600 S Woodland Blvd,28.983689,-81.300079,3/15/2011,32720
11,ELEC,Delray Nissan,2200 S Federal Hwy,26.435016,-80.072333,3/15/2011,33483


In [25]:
#change datatype on zip column & Open Date column
stat_df['ZIP'] = stat_df['ZIP'].astype(int)
stat_df['Open Date'] =pd.to_datetime(stat_df['Open Date'])
stat_df.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,Open Date,ZIP
6,ELEC,City of Delray Beach - Banker's Row Parking Lot,200 NE 1st Ave,26.465314,-80.072190,2011-06-15,33444
8,ELEC,Firkins Nissan,1611 Cortez Rd W,27.462023,-82.578036,2011-03-15,34207
9,ELEC,Maus Nissan,937 S Suncoast Blvd,28.848733,-82.579470,2011-03-15,34448
10,ELEC,Deland Nissan,2600 S Woodland Blvd,28.983689,-81.300079,2011-03-15,32720
11,ELEC,Delray Nissan,2200 S Federal Hwy,26.435016,-80.072333,2011-03-15,33483


In [52]:
#import Zip code to county Date to merge it with main DF
zip_df = "static/data/ZIP-COUNTY-FIPS_2018-03.csv"
zip_df = pd.read_csv(zip_df)

In [27]:
#remove unecessary columns
zip_df = zip_df[['ZIP', 'COUNTYNAME']]
zip_df.head()

,ZIP,COUNTYNAME
0,36091,Autauga County
1,36758,Autauga County
2,36006,Autauga County
3,36067,Autauga County
4,36701,Autauga County


In [48]:
#combine Stations & counties Df into one 
combined_df = pd.merge(stat_df, zip_df, on='ZIP')
combined_df.sort_values('COUNTYNAME')

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,Open Date,ZIP,COUNTYNAME
645,ELEC,La Quinta Inn by Wyndham Gainesville - Tesla D...,920 NW 69th Ter,29.661692,-82.417592,2019-09-30,32605,Alachua County
630,ELEC,Hampton Inn - Gainesville,4225 SW 40th Blvd,29.614982,-82.382567,2020-11-13,32608,Alachua County
205,ELEC,Magnolia Parke Square,NW 39th Pl and NW 48 Terrace,29.689550,-82.395100,2015-03-01,32606,Alachua County
629,ELEC,Butler Town Center - Tesla Supercharger,3217 Southwest 35th Boulevard,29.624400,-82.376800,2019-09-30,32608,Alachua County
628,ELEC,Fairfield Inn & Suites Gainesville - Tesla Des...,3877 SW 37th Blvd,29.618709,-82.378398,2019-03-01,32608,Alachua County
...,...,...,...,...,...,...,...,...
691,ELEC,Best Western - Niceville,106 Bayshore Dr,30.520782,-86.490488,2020-11-13,32578,Walton County
693,ELEC,Holiday Inn Express & Suites - Niceville,410 W John Sims Pkwy,30.522455,-86.501382,2020-11-17,32578,Walton County
415,ELEC,TOPSL Commercial Center - Tesla Destination,8955 Emerald Coast Pkwy,30.377886,-86.325616,2020-11-03,32550,Walton County
411,ELEC,Residence Inn Marriott Sandestin at Grand Boul...,300 Grand Blvd,30.379551,-86.318439,2017-09-01,32550,Walton County


In [50]:
#create a shorter datafram for the last two years of stations
combined_df = combined_df[(combined_df['Open Date']> "2018-01-01") & (combined_df['Open Date']< "2020-12-31")]
two_years_df = combined_df.sort_values('COUNTYNAME')
two_years_df.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,Open Date,ZIP,COUNTYNAME
630,ELEC,Hampton Inn - Gainesville,4225 SW 40th Blvd,29.614982,-82.382567,2020-11-13,32608,Alachua County
629,ELEC,Butler Town Center - Tesla Supercharger,3217 Southwest 35th Boulevard,29.624400,-82.376800,2019-09-30,32608,Alachua County
628,ELEC,Fairfield Inn & Suites Gainesville - Tesla Des...,3877 SW 37th Blvd,29.618709,-82.378398,2019-03-01,32608,Alachua County
645,ELEC,La Quinta Inn by Wyndham Gainesville - Tesla D...,920 NW 69th Ter,29.661692,-82.417592,2019-09-30,32605,Alachua County
694,ELEC,30 Avenue - Tesla Destination,12805 U.S. 98,30.280113,-86.009869,2020-11-03,32461,Bay County
